# CAPSTONE PROJECT

## Week 3: Part 1

## Required installations

In [1]:
!pip install bs4
!pip install lxml
!pip install requests
print('Instalations completed')

Instalations completed


## Extracting data from Wikipedia

In [32]:
import pandas
import requests
from bs4 import BeautifulSoup
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
url_clean=requests.get(url).text

soup=BeautifulSoup(url_clean,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
    

## Creating DataFrame with Pandas

In [33]:
table_rows = table.find_all('tr')
data = []
for row in table_rows:
    td=[]
    for t in row.find_all('td'):
        td.append(t.text.strip())
    data.append(td)
df = pandas.DataFrame(data, columns=['Postal Code', 'Borough', 'Neighbourhood'])

## Cleaning & Grouping Dataframe

In [35]:
# Cleaning 'Not assigned' rows
df = df[~df['Borough'].isnull()]
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df = df.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()
df['Neighbourhood'].replace('Not assigned',df['Borough'],inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Result & Shape


In [37]:
df.describe()

,Postal Code,Borough,Neighbourhood
count,103,103,103
unique,103,10,98
top,M6A,North York,Downsview
freq,1,24,4


In [38]:
df.head(14)

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West
9,M1N,Scarborough,Birch Cliff / Cliffside West


In [39]:
df.shape

(103, 3)

# Latitude & Longuitude

In [42]:
url_geo='http://cocl.us/Geospatial_data'
df_geo_coordinate = pd.read_csv(url_geo)
df_geo_coordinate.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merge

In [43]:
df_merged = df.join(df_geo_coordinate.set_index('Postal Code'), on='Postal Code')
df_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
